<a href="https://colab.research.google.com/github/davinhill/Convolution_Captioning/blob/master/embed/Create_word_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [0]:
import os

# Path where the data should be located
path = '/content/drive/My Drive/Colab Notebooks/IE534_ImageCaptioning/Data'
os.chdir(path)

In [5]:
from pycocotools.coco import COCO 

import torch
from torchvision import datasets, transforms
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import io
import itertools
import nltk
nltk.download('punkt')

import os
from datetime import datetime


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
# Load Captions from Annotation File

##################
# Validation Set
##################
# Load Annotations
cap = COCO(os.path.join(path, 'annotations/captions_val2017.json'))

# Create list of captions
ann_ids = cap.getAnnIds(imgIds = [])
ann_list = cap.loadAnns(ids = ann_ids)

cap_val_raw = []
for dict in ann_list:
  cap_val_raw.append(dict['caption'])

##################
# Train Set
##################
# Load Annotations
cap = COCO(os.path.join(path, 'annotations/captions_train2017.json'))

# Create list of captions
ann_ids = cap.getAnnIds(imgIds = [])
ann_list = cap.loadAnns(ids = ann_ids)

cap_train_raw = []
for dict in ann_list:
  cap_train_raw.append(dict['caption'])

loading annotations into memory...
Done (t=0.83s)
creating index...
index created!
loading annotations into memory...
Done (t=3.09s)
creating index...
index created!


In [7]:
# Number of Captions:
len(cap_train_raw)

591753

In [8]:
# Tokenize Caption List

cap_val = []
for caption in cap_val_raw:
  line = nltk.word_tokenize(caption)
  line = [w.lower() for w in line] 
  cap_val.append(line)


cap_train = []
for caption in cap_train_raw:
  line = nltk.word_tokenize(caption)
  line = [w.lower() for w in line] 
  cap_train.append(line)


# summarize token statistics
x = []
for tokens in cap_train:
    x.append(len(tokens))
x = np.asarray(x)
print('Total: ', np.sum(x), ' Min: ', np.min(x), ' Max: ', np.max(x), ' Mean: ', np.mean(x), ' Std: ', np.std(x), ' Med: ', np.median(x))



Total:  6687792  Min:  6  Max:  57  Mean:  11.301661335050266  Std:  2.596305429474608  Med:  11.0


In [0]:
# create word_to_id and id_to_word translations
# word_to_id is a dictionary
# id_to_word is a np array

all_tokens = itertools.chain.from_iterable(cap_train)
word_to_id = {token: idx for idx, token in enumerate(set(all_tokens))}


all_tokens = itertools.chain.from_iterable(cap_train)
id_to_word = np.asarray([token for idx, token in enumerate(set(all_tokens))])

In [0]:
## sort by word frequency

token_ids = [[word_to_id[token] for token in x] for x in cap_train]
count = np.zeros(id_to_word.shape)

for x in token_ids:
    for token in x:
        count[token] += 1
        
indices = np.argsort(-count)
id_to_word = id_to_word[indices]
count = count[indices]


## recreate word_to_id based on sorted list
word_to_id = {token: (idx+4) for idx, token in enumerate(id_to_word)}

# add start/end/unknown token
word_to_id['<S>'] = 1
word_to_id['</S>'] = 2
word_to_id['UNK'] = 3

# add start/end/unknown token
id_to_word = np.insert(id_to_word, 0, 'UNK')
id_to_word = np.insert(id_to_word, 0, '</S>')
id_to_word = np.insert(id_to_word, 0, '<S>')
id_to_word = np.insert(id_to_word, 0, '<MASK>')



In [11]:
# test

id_to_word[word_to_id.get('dog')]

id_to_word[18]

','

In [0]:
# save to file


os.chdir('/content/drive/My Drive/Colab Notebooks/IE534_ImageCaptioning/')

# save word_to_id
import pickle
with open('word_to_id.p', 'wb') as fp:
    pickle.dump(word_to_id, fp, protocol=4)

## save id_to_word
np.save('id_to_word.npy',np.asarray(id_to_word))
